In [1]:
%run stdPackages.ipynb
os.chdir(d['py'])
import US_main, US_c, US_policy
os.chdir(d['curr'])

## US FH:

Define data and parameters:

In [2]:
dates_log = pd.Index([1950, 1980, 2010, 2040, 2070, 2100], name = 't')
ν_log = np.array([1.504840069, 1.394563144, 1.178137696, 1.055068673, 1.018706685, 1.018706685])
T_log = len(ν_log)
T = T_log+3
dates = dates_log.union(pd.Index([dates_log[-1]+30*i for i in range(1,T-T_log+1)]))
ν = np.hstack([ν_log, np.full(T-T_log, ν_log[-1])])
A = np.ones(T) # normalize exog. productivity at 1 
t0date = 2010 # calibration date
t0 = dates.get_loc(t0date) # index for year of calibration 


# "Simple" calibration:
α = .281 # capital income share

# Household types:
γj  = np.array([.056, 0.449, 0.307, 0.188])
ni = len(γj)-1
hours = np.array([1415.38, 2114.79, 2315.83, 2458.91]) # hours
wagerates = np.array([6.914, 14.88, 27.32, 62.26]) # wages
income = hours*wagerates
zxj = hours/hours[1:].mean()
zηj = income/income[1:].mean()
pj = np.array([0.385, 0.55, 0.652, 0.74])
μj = np.array([0.362, 0.526, 0.684, 0.756]) # voter turnout 


# other targets:
τ0 = .158 # target labor tax rate
RR = 39.4/50.1 # replacement rate
universalShare = 3.4/15.8 # share of ss going to universal
R0 = 2.443 


# Initial/baseline values:
ξ0 = .1
ρ0 = 1.1
ω0 = 1.5
# βj = np.ones(ni+1)

Package them in kwargs:

In [3]:
kwargs = {'α': α, 'A': A, 'ν': ν, 'γj': γj, 'zxj': zxj, 'zηj': zηj, 'pj': pj, 'μj': μj, 'τ0': τ0, 'RR0': RR, 'UShare0': universalShare, 'R0': R0, 'ξ': ξ0, 'ρ': ρ0, 'ω': ω0}

Initialize model:

In [4]:
m = US_main.Model(ni = ni, T = T, gridkwargs = {'glob_ns': 50}, **kwargs)

Try the following: Calibrate with PEE, calibrate ESC_FH with x0_from_solp = true in the first loop. Then disable it and rely on x0 in later runs.

In [5]:
solρ = dict.fromkeys(np.round(np.linspace(ρ0, ρ0+1, 11), 1))
polρ = dict.fromkeys(np.round(np.linspace(ρ0, ρ0+1, 11), 1))

In [ ]:
m.USCalSimple_PEE_FH(t0);
par0 = m.USCalSimple_ESC_FH(t0)
solρ[ρ0], polρ[ρ0] = m.ESC_FH(returnPols = True)
# m.ESC.kwargs_t = {'style': 'Grid'} | m.ESC.grids
m.ESC.kwargs_t = {'style': 'Vector', 'x0_from_solp': False} # Try not using x0_from_solp 
for ρi in list(solρ.keys())[1:7]:
    m.db['ρ'] = ρi
    m.USCalSimple_ESC_FH(t0);
    solρ[ρi], polρ[ρi] = m.ESC_FH(returnPols = True)
# Switch to grid search for the final terms (this is where things get more tricky):
m.ESC.kwargs_t = {'style': 'Grid'} | m.ESC.grids
for ρi in list(solρ.keys())[7:]:
    m.db['ρ'] = ρi
    m.USCalSimple_ESC_FH(t0);
    solρ[ρi], polρ[ρi] = m.ESC_FH(returnPols = True)

Now turn $\rho$ towards 0.5 instead:

In [ ]:
solρ_lower = dict.fromkeys(np.round(np.linspace(.9, .5, 5), 1))
polρ_lower = dict.fromkeys(np.round(np.linspace(.9, .5, 5), 1))

In [ ]:
kwargs['ρ'] = 0.9
kwargs['ω'] = 2
m = US_main.Model(ni = ni, T = T, gridkwargs = {'glob_ns': 50}, **kwargs)

In [ ]:
m.USCalSimple_PEE_FH(t0);
m.USCalSimple_ESC_FH(t0);
m.ESC.kwargs_t = {'style': 'Vector', 'x0_from_solp': False} # Try not using x0_from_solp 
for ρi in solρ_lower:
    m.db['ρ'] = ρi
    m.USCalSimple_ESC_FH(t0);
    solρ_lower[ρi], polρ_lower[ρi] = m.ESC_FH(returnPols = True)